# Uplift Modeling with EconML using MovieLens 1M
This notebook downloads MovieLens 1M data, simulates treatment and renewal outcomes, introduces missing data, imputes values, and trains S-, T-, and X-Learners using different base learners.

In [51]:
!pip uninstall  econml scikit-learn pandas numpy

Found existing installation: econml 0.15.1
Uninstalling econml-0.15.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/econml-0.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/econml/*
Proceed (Y/n)? Y
  Successfully uninstalled econml-0.15.1
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.5.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    

In [52]:
!pip  install --no-cache-dir  econml scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 172.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 162.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 178.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 152.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 246.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 272.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 282.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from econml.metalearners import SLearner, TLearner, XLearner

In [2]:
# Download and extract MovieLens 1M dataset
!pip install wget
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

--2025-05-18 22:12:33--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  5.88MB/s    in 1.0s    

2025-05-18 22:12:34 (5.88 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [3]:
# Download and extract MovieLens 1M dataset
# The wget and unzip commands appear to be working correctly based on your output.
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

# Add checks to verify if the directory and file exist
import os

# Correct the path to reflect the nested directory structure
if os.path.exists('ml-1m/ml-1m/ratings.dat'):
    print("ml-1m/ml-1m/ratings.dat found. Proceeding to load data.")
else:
    print("Error: ml-1m/ml-1m/ratings.dat not found. Please check the extraction path.")
    # If the file is still not found after correcting the path, there might be
    # a deeper issue with the unzip process or disk.
    # import sys
    # sys.exit(1) # Uncomment to exit the notebook execution if the file is not found

--2025-05-18 22:12:35--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.3’

ml-1m.zip.3         100%[===================>]   5.64M  5.93MB/s    in 1.0s    

2025-05-18 22:12:36 (5.93 MB/s) - ‘ml-1m.zip.3’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   
ml-1m/ml-1m/ratings.dat found. Proceeding to load data.


In [4]:
# Load data
ratings = pd.read_csv('ml-1m//ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
#movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
#                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [5]:
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [6]:
# merge ratings, users, and movies
df = ratings.merge(users, on = 'UserID').merge(movies, on = 'MovieID')
df.sample(10)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
487757,3001,2058,4,970619239,M,18,4,94704,"Negotiator, The (1998)",Action|Thriller
432530,2638,2401,3,974177402,M,25,14,06405,Pale Rider (1985),Western
630116,3809,3702,3,965962677,M,35,6,20650,Mad Max (1979),Action|Sci-Fi
278183,1677,2720,1,974709408,M,18,14,97123,Inspector Gadget (1999),Action|Adventure|Children's|Comedy
994147,6003,1845,4,956983359,F,45,17,78722,Zero Effect (1998),Comedy|Thriller
196682,1207,2599,3,974845546,M,50,0,43617,Election (1999),Comedy
59755,406,2734,1,976291421,M,25,20,55105,"Mosquito Coast, The (1986)",Drama
461485,2847,3809,2,1028961591,M,18,4,10027,What About Bob? (1991),Comedy
307299,1835,3432,2,974879639,M,25,19,11501,Death Wish 3 (1985),Action|Drama
322723,1912,1982,4,974829155,M,25,4,91325,Halloween (1978),Horror


In [7]:
# Feature creation
np.random.seed(42)
df['WatchTime'] = df['Rating'] * np.random.uniform(15, 30, size=len(df)).astype(int)


In [8]:
df.sample(2)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime
954034,5759,1732,5,959559219,F,25,1,08904,"Big Lebowski, The (1998)",Comedy|Crime|Mystery|Thriller,145
184522,1147,1238,3,974873713,M,25,20,98101,Local Hero (1983),Comedy,60


In [9]:
df['Timestamp_Date'] = pd.to_datetime(df['Timestamp'], unit='s').dt.strftime('%Y-%m-%d')

In [ ]:
df.sample(3)

In [10]:
df['TenureMonths'] = (df['Timestamp'] - df['Timestamp'].min()) // (60*60*24*30)

In [ ]:
df.sample(3)

In [11]:
#regenerate ages with randome integers between 18-69
df2 = pd.DataFrame()
df2['UserID'] = df['UserID'].drop_duplicates()
df2.head()

,UserID
0,1
53,2
182,3
233,4
254,5


In [12]:
df2['Age'] = np.random.randint(18, 70, df2.shape[0])
df2.head()

,UserID,Age
0,1,60
53,2,32
182,3,25
233,4,39
254,5,28


In [13]:
df_user = df.merge(df2, on = 'UserID', how = 'left')
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age_x,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age_y
902831,5458,3176,4,965532483,F,18,2,98102,"Talented Mr. Ripley, The (1999)",Drama|Mystery|Thriller,64,2000-08-06,3,30
246610,1490,2405,4,974771799,M,50,7,90505,"Jewel of the Nile, The (1985)",Action|Adventure|Comedy|Romance,112,2000-11-21,6,24
685505,4097,2953,2,965415862,M,25,7,63366,Home Alone 2: Lost in New York (1992),Children's|Comedy,42,2000-08-04,3,32
379340,2216,3910,2,974600401,F,25,20,60612,Dancer in the Dark (2000),Drama|Musical,48,2000-11-19,6,65
218083,1322,305,5,974778510,M,56,0,94043,Ready to Wear (Pret-A-Porter) (1994),Comedy,100,2000-11-21,6,28


In [14]:
df_user = df_user.drop('Age_x', axis= 1).rename(columns = {'Age_y':'Age'})
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
608684,3691,2571,5,966308453,M,7,21030,"Matrix, The (1999)",Action|Sci-Fi|Thriller,120,2000-08-15,3,54
467356,2880,1372,1,989097892,M,12,55405,Star Trek VI: The Undiscovered Country (1991),Action|Adventure|Sci-Fi,22,2001-05-05,12,57
433317,2641,1203,5,986741590,M,6,19428,12 Angry Men (1957),Drama,90,2001-04-08,11,63
813024,4880,2677,5,962763380,F,9,95030,Buena Vista Social Club (1999),Documentary,105,2000-07-05,2,32
2366,19,3552,3,978556909,M,10,48073,Caddyshack (1980),Comedy,48,2001-01-03,8,48


In [ ]:
df_user.shape

In [15]:
# Feature creation
user_features = df_user.groupby('UserID').agg({
    'WatchTime': 'sum',
    'MovieID': 'nunique',
    'TenureMonths': 'max',
    'Age': 'first',
    'Occupation': 'first'
}).rename(columns={'WatchTime': 'TotalWatchTime', 'MovieID': 'UniqueMovies'})

In [ ]:
user_features.sample(5)

In [16]:
# Introduce and impute missing data
user_features.loc[user_features.sample(frac=0.1).index, 'TotalWatchTime'] = np.nan
user_features.sample(10)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
5625,13174.0,152,1,21,1
2884,1865.0,20,6,18,3
112,NaN,60,8,22,16
5395,9930.0,121,9,28,1
3711,8001.0,88,3,62,0
2958,2732.0,43,5,44,7
637,30417.0,377,7,28,12
1956,4512.0,65,6,28,0
809,8088.0,94,7,53,17


In [ ]:
user_features['TotalWatchTime'].isnull().sum()

In [17]:
user_features.loc[user_features.sample(frac=0.1).index, 'TenureMonths'] = np.nan
user_features.sample(15)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
5221,2676.0,30,1.0,24,1
1803,21521.0,284,NaN,46,2
3972,1984.0,25,NaN,45,6
5112,NaN,317,2.0,34,13
5827,8709.0,109,NaN,39,2
5046,69522.0,868,33.0,27,16
3360,4556.0,58,4.0,47,0
2963,3520.0,39,NaN,62,4
2255,13760.0,175,7.0,54,6


In [18]:
# Introduce and impute missing data
user_features['TotalWatchTime'] =user_features['TotalWatchTime'].fillna(user_features['TotalWatchTime'].median())
user_features['TenureMonths']= user_features['TenureMonths'].fillna(user_features['TenureMonths'].median())

In [19]:
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()

In [ ]:
engaged.head()

In [ ]:
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
uplift.head(2)

In [20]:
# Simulate treatment and renewal
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
user_features['renewed'] = np.random.binomial(1, base_rate + uplift)
X = user_features[['TenureMonths', 'TotalWatchTime', 'UniqueMovies']]
T = user_features['treatment'].values
Y = user_features['renewed'].values

In [ ]:
T.shape, type(T), T

In [21]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(2)

In [22]:
X_train.select_dtypes(include=['number']).columns

Index(['TenureMonths', 'TotalWatchTime', 'UniqueMovies'], dtype='object')

In [ ]:
#feature scaling
# # Preprocessing
# numeric_features = ["tenure_months", "prior_engagement_score", "weekly_watch_hours", "num_devices"]
# categorical_features = ["device_type", "payment_method", "account_type", "region", "has_kids_profile", "promo_eligible"]

# preprocessor = ColumnTransformer([
#     ("num", StandardScaler(), numeric_features),
#     ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
# ])

# # Fit and transform
# X_train_proc = preprocessor.fit_transform(X_train)
# X_test_proc = preprocessor.transform(X_test)
# # 🎯 Evaluate both
# print("Sklearn GBM:")
# print(classification_report(y_test, sk_gbm.predict(X_test_proc)))
# print("AUC:", roc_auc_score(y_test, sk_gbm.predict_proba(X_test_proc)[:, 1]))


In [23]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

In [24]:
numeric_features  = X_train.select_dtypes(include=['number']).columns.tolist()
#cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features)
 #   ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

# Fit and transform
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

In [25]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': [ 'liblinear'],
    'class_weight': [None, 'balanced'],
    'max_iter': [100, 200,500,1000]
    }

lr_grid = GridSearchCV(LogisticRegression(), param_grid, cv=3)
lr_grid.fit(X_train_proc, Y_train)
best_lr = lr_grid.best_estimator_

In [ ]:

lr_grid.best_score_, lr_grid.best_params_

In [ ]:
best_lr

In [26]:
# Train learners
from econml.metalearners import SLearner, TLearner, XLearner # Re-import the learners
s_learner = SLearner(overall_model=best_lr)
s_learner.fit(Y_train, T_train, X=X_train_proc)
s_te = s_learner.effect(X_test_proc)
pd.DataFrame({'S_Learner': s_te}).head()

,S_Learner
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [44]:
s_te.max()

0.0

In [ ]:
# param_grid = {
#     'C': [0.01, 0.1, 1, 10, 100],
#     'penalty': ['l1', 'l2', 'elasticnet'],
#     'solver': [ 'saga'],
#     'class_weight': [None, 'balanced'],
#     'max_iter': [10000, 20000,50000],
#      'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
# }

# lr_grid = GridSearchCV(LogisticRegression(), param_grid, cv=3)
# lr_grid.fit(X_train_proc, Y_train)
# best_lr = lr_grid.best_estimator_

In [27]:
# Hyperparameter tuning
# lr_grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid={'C': [0.01, 0.1, 1, 10]}, cv=3)
# lr_grid.fit(X_train, Y_train)
# best_lr = lr_grid.best_estimator_

rf_random = RandomizedSearchCV(RandomForestRegressor(random_state=42),
    param_distributions={'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    n_iter=4, cv=3, random_state=42)
rf_random.fit(X_train, Y_train)
best_rf = rf_random.best_estimator_

gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42),
    param_grid={'n_estimators': [100, 150], 'learning_rate': [0.05, 0.1]}, cv=3)
gb_grid.fit(X_train, Y_train)
best_gb = gb_grid.best_estimator_

In [28]:
#now optimize the model for s-learner where the treatment/control label will be part of the feature set
# Add treatment flag as a feature for S-Learner
X_train_proc_s = np.hstack([X_train_proc, T_train.reshape(-1, 1)])
X_test_proc_s = np.hstack([X_test_proc, T_test.reshape(-1, 1)])# Now do hyperparameter tuning on X_train_with_treat, Y_train
lr_grid_s = GridSearchCV(LogisticRegression(), param_grid, cv=3)
lr_grid_s.fit(X_train_proc_s, Y_train)
best_lr_s = lr_grid_s.best_estimator_


X_train_s = np.hstack([X_train, T_train.reshape(-1, 1)])
X_test_s = np.hstack([X_test, T_test.reshape(-1, 1)])# Now do hyperparameter tuning on X_train_with_treat, Y_train
rf_random_s = RandomizedSearchCV(RandomForestRegressor(random_state=42),
    param_distributions={'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    n_iter=4, cv=3, random_state=42)
rf_random_s.fit(X_train_s, Y_train)
best_rf_s = rf_random_s.best_estimator_

gb_grid_s = GridSearchCV(GradientBoostingRegressor(random_state=42),
    param_grid={'n_estimators': [100, 150], 'learning_rate': [0.05, 0.1]}, cv=3)
gb_grid_s.fit(X_train_s, Y_train)
best_gb_s = gb_grid_s.best_estimator_


In [30]:
# Train learners
#logistic regression as base learners
s_learner = SLearner(overall_model = best_lr_s)
t_learner = TLearner(models = best_lr)
x_learner = XLearner(models = best_lr)
s_learner.fit(Y_train, T_train, X=X_train_proc)
t_learner.fit(Y_train, T_train, X=X_train_proc)
x_learner.fit(Y_train, T_train, X=X_train_proc)
s_te = s_learner.effect(X_test_proc)
t_te = t_learner.effect(X_test_proc)
x_te = x_learner.effect(X_test_proc)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head()

,S_Learner,T_Learner,X_Learner
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [31]:
# Train learners
# random forest as base learners
s_learner = SLearner(overall_model = best_rf_s)
t_learner = TLearner(models = best_rf)
x_learner = XLearner(models = best_rf)
s_learner.fit(Y_train, T_train, X=X_train)
t_learner.fit(Y_train, T_train, X=X_train)
x_learner.fit(Y_train, T_train, X=X_train)
s_te = s_learner.effect(X_test)
t_te = t_learner.effect(X_test)
x_te = x_learner.effect(X_test)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head()
#pd.DataFrame({ 'T_Learner': t_te, 'X_Learner': x_te}).head()

,S_Learner,T_Learner,X_Learner
0,-0.002594,0.105159,0.036507
1,-0.026122,-0.006407,0.030765
2,0.017428,0.028865,0.017686
3,0.016547,-0.013323,-0.015395
4,0.017782,-0.020370,0.097980


In [47]:
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head(10)

,S_Learner,T_Learner,X_Learner
0,0.295050,0.364771,0.167604
1,0.347505,0.289605,0.286442
2,0.046786,0.034559,0.051505
3,0.007682,-0.011223,0.014688
4,0.053603,0.069279,0.056319
5,0.023310,0.013461,0.027211
6,0.002128,0.019070,0.037663
7,0.358307,0.360735,0.369031
8,0.180427,0.140381,0.108202
9,-0.094170,-0.067629,-0.031057


In [32]:
#gradient boosting as base learners
s_learner = SLearner(overall_model = best_gb_s)
t_learner = TLearner(models = best_gb)
x_learner = XLearner(models = best_gb)
s_learner.fit(Y_train, T_train, X=X_train)
t_learner.fit(Y_train, T_train, X=X_train)
x_learner.fit(Y_train, T_train, X=X_train)
s_te = s_learner.effect(X_test)
t_te = t_learner.effect(X_test)
x_te = x_learner.effect(X_test)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head(10)

,S_Learner,T_Learner,X_Learner
0,0.026510,-0.021457,0.005828
1,0.187264,0.108285,0.133065
2,0.005841,0.007394,0.007086
3,0.005841,-0.000005,0.004645
4,0.126918,0.123982,0.102498
5,0.007080,0.040932,0.016941
6,-0.000420,0.011843,0.008007
7,0.093927,-0.000431,0.083771
8,0.046334,0.060504,0.049504
9,0.051793,0.080404,0.078845
